<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0"> </div>
    <div style="float: left; margin-left: 10px;"> <h1>LangChain for Generative AI</h1>
<h1>ChatBot</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

import langchain

from langchain.prompts import PromptTemplate
from langchain.document_loaders import GutenbergLoader

from langchain.memory import ConversationBufferMemory
from langchain.memory.chat_message_histories.in_memory import ChatMessageHistory

from langchain.schema import messages_from_dict, messages_to_dict

from langchain.agents import Tool
from langchain.agents import initialize_agent
from langchain.agents import AgentType

from langchain.chains import LLMChain, ConversationalRetrievalChain, ConversationChain
from langchain.chains import RetrievalQA

from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

import langchain_openai
from langchain_openai import ChatOpenAI

import tempfile

import watermark

%load_ext watermark
%matplotlib inline

We start by print out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.13.3
IPython version      : 9.2.0

Compiler    : Clang 17.0.0 (clang-1700.0.13.3)
OS          : Darwin
Release     : 25.2.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Git hash: 2563372cfdc6863bbd75970c20a0f7113c7e3c80

langchain_core          : 0.3.62
watermark               : 2.5.0
langchain_text_splitters: 0.3.8
numpy                   : 2.2.5
langchain_openai        : 0.3.18
matplotlib              : 3.10.3
langchain_community     : 0.3.24
pandas                  : 2.2.3
langchain               : 0.3.25



Load default figure style

In [3]:
plt.style.use('d4sci.mplstyle')

# Start

In [4]:
cache_dir = "./cache"

In [5]:
loader = GutenbergLoader(
    "https://www.gutenberg.org/cache/epub/1513/pg1513.txt"
)

document = loader.load()

extrait = ' '.join(document[0].page_content.split()[:100])
display(extrait + " .......")

'The Project Gutenberg eBook of Romeo and Juliet This ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this ebook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using this eBook. Title: Romeo and Juliet Author: William Shakespeare Release date: November .......'

In [6]:
len(document[0].page_content)

178720

In [7]:
text_splitter = CharacterTextSplitter(
    chunk_size=1024, # Each chunk is of size 1024
    chunk_overlap=64 # Neigboring chunks overlap by 64 characters
) 

texts = text_splitter.split_documents(document)

In [8]:
print(texts[10].page_content)

I drew to part them, in the instant came


The fiery Tybalt, with his sword prepar’d,


Which, as he breath’d defiance to my ears,


He swung about his head, and cut the winds,


Who nothing hurt withal, hiss’d him in scorn.


While we were interchanging thrusts and blows


Came more and more, and fought on part and part,


Till the Prince came, who parted either part.





LADY MONTAGUE.


O where is Romeo, saw you him today?


Right glad I am he was not at this fray.





BENVOLIO.


Madam, an hour before the worshipp’d sun


Peer’d forth the golden window of the east,


A troubled mind drave me to walk abroad,


Where underneath the grove of sycamore


That westward rooteth from this city side,


So early walking did I see your son.


Towards him I made, but he was ware of me,


And stole into the covert of the wood.


I, measuring his affections by my own,


Which then most sought where most might not be found,


Being one too many by my weary self,


In [9]:
print(texts[11].page_content)

Being one too many by my weary self,


Pursu’d my humour, not pursuing his,


And gladly shunn’d who gladly fled from me.





MONTAGUE.


Many a morning hath he there been seen,


With tears augmenting the fresh morning’s dew,


Adding to clouds more clouds with his deep sighs;


But all so soon as the all-cheering sun


Should in the farthest east begin to draw


The shady curtains from Aurora’s bed,


Away from light steals home my heavy son,


And private in his chamber pens himself,


Shuts up his windows, locks fair daylight out


And makes himself an artificial night.


Black and portentous must this humour prove,


Unless good counsel may the cause remove.





BENVOLIO.


My noble uncle, do you know the cause?





MONTAGUE.


I neither know it nor can learn of him.





BENVOLIO.


Have you importun’d him by any means?





MONTAGUE.


Both by myself and many other friends;


But he, his own affections’ counsellor,


Is to himself—I will not say how true—


In [10]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"

embeddings = HuggingFaceEmbeddings(
    model_name=model_name, 
    cache_folder=cache_dir
)  # Use a pre-cached model

/var/folders/lr/j1bs1q851k15cj5y777nxwph0000gn/T/ipykernel_16930/3001161102.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [11]:
vectordb = Chroma.from_documents(
    texts, 
    embeddings, 
    persist_directory=cache_dir
)

In [12]:
question = "Romeo!"

docs = vectordb.similarity_search(question, k=2)

In [13]:
# Check the length of the document
print(len(docs))

2


In [14]:
# Check the content of the first document
print(docs[0].page_content)
print("="*20)
print(docs[1].page_content)

Romeo! My cousin Romeo! Romeo!





MERCUTIO.


He is wise,


And on my life hath stol’n him home to bed.





BENVOLIO.


He ran this way, and leap’d this orchard wall:


Call, good Mercutio.





MERCUTIO.


Nay, I’ll conjure too.
Romeo! My cousin Romeo! Romeo!





MERCUTIO.


He is wise,


And on my life hath stol’n him home to bed.





BENVOLIO.


He ran this way, and leap’d this orchard wall:


Call, good Mercutio.





MERCUTIO.


Nay, I’ll conjure too.


Create a wrapper around the functionality of our vector database so we can search for similar documents in the vectorstore

In [15]:
retriever = vectordb.as_retriever()

In [16]:
llm = ChatOpenAI(model='gpt-4o', temperature=0)

In [17]:
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum. Keep the answer as concise as possible. 
Always say "thanks for asking!" at the end of the answer. 

{context}

Question: {question}

Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [18]:
pprint(QA_CHAIN_PROMPT.template)

('Use the following pieces of context to answer the question at the end. \n'
 "If you don't know the answer, just say that you don't know, don't try to "
 'make up an answer. \n'
 'Use three sentences maximum. Keep the answer as concise as possible. \n'
 'Always say "thanks for asking!" at the end of the answer. \n'
 '\n'
 '{context}\n'
 '\n'
 'Question: {question}\n'
 '\n'
 'Helpful Answer:')


In [19]:
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    retriever=retriever,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [20]:
query = "What is Juliets family?"

query_results_venice = qa.invoke(query)
print("#" * 12)
query_results_venice['result']

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


############


"Juliet's family is the Capulets. Thanks for asking!"

In [21]:
query = "What happens to Romeo and Juliet?"
query_results_romeo = qa.invoke(query)
print("#" * 12)
query_results_romeo['result']

############


'Romeo is exiled for killing Tybalt, and Juliet is devastated by the news. Their separation and a series of tragic misunderstandings ultimately lead to both Romeo and Juliet taking their own lives. Thanks for asking!'

In [22]:
query = "Who is Mercutio?"
query_results_romeo = qa.invoke(query)
print("#" * 12)
query_results_romeo['result']

############


'Mercutio is a character in William Shakespeare\'s play "Romeo and Juliet." He is a close friend of Romeo and is known for his witty and playful nature. Thanks for asking!'

In [23]:
query = "Does Romeo live?"
qa_chain_docs = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       
                                       # Return source documents
                                       return_source_documents=True,
                                       
                                            chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


result = qa_chain_docs({"query": question})
result["result"]

/var/folders/lr/j1bs1q851k15cj5y777nxwph0000gn/T/ipykernel_16930/948548302.py:11: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain_docs({"query": question})


'It seems like you are quoting from a scene in "Romeo and Juliet" where Benvolio and Mercutio are looking for Romeo after he has leapt over the orchard wall. They believe he has gone home to bed, but he is actually hiding nearby. Thanks for asking!'

In [24]:
len(result['source_documents'])

4

In [25]:
print(result['source_documents'][1].page_content)

Romeo! My cousin Romeo! Romeo!





MERCUTIO.


He is wise,


And on my life hath stol’n him home to bed.





BENVOLIO.


He ran this way, and leap’d this orchard wall:


Call, good Mercutio.





MERCUTIO.


Nay, I’ll conjure too.


In [26]:
result = llm.invoke("Does Romeo live in Romeo and Juliet?")

In [27]:
result.content

'No, Romeo does not live in "Romeo and Juliet." In William Shakespeare\'s tragic play, both Romeo and Juliet die. Romeo takes his own life after mistakenly believing that Juliet is dead, and Juliet subsequently takes her own life upon finding Romeo dead. Their deaths ultimately reconcile their feuding families, the Montagues and the Capulets.'

<center>
     <img src="data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>